In [15]:
import requests

r = requests.get('https://www.eia.gov/electricity/state/index.php')

In [16]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'html.parser')

In [17]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   State Electricity Profiles - Energy Information Administration
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="en-us" http-equiv="Content-Language"/>
  <meta content="all" name="robots"/>
  <meta content="EIA - Energy Information Administration" name="agency"/>
  <meta content="official energy statistics, data, analysis and forecasting" name="subject"/>
  <meta content="Energy Information Administration - EIA - Official Energy Statistics from the U.S. Government" name="Description"/>
  <link href="/global/styles/screen.css" media="screen, projection" rel="stylesheet" type="text/css"/>
  <link href="/global/styles/eia-styles.min.css?v=2.0" media="screen, projection" rel="st

In [18]:
len(soup.find_all('a'))


288

In [19]:
for i in soup.find_all('a'):
    print(i.get('href'))

https://twitter.com/EIAgov
None
https://www.facebook.com/eiagov
None
#page-sub-nav
/
#
#menu
#menu
#menu
#menu
#menu
#menu
#menu
#nav-sources
#nav-topics
#nav-geography
#nav-tools
#nav-learn
#nav-news
/petroleum/
/petroleum/weekly/
/petroleum/supply/weekly/
/naturalgas/
http://ir.eia.gov/ngs/ngs.html
/naturalgas/weekly/
/electricity/
/electricity/monthly/
/electricity/data/browser/
/consumption/
/consumption/residential/
/consumption/commercial/
/coal/
/coal/production/quarterly/
/beta/coal/data/browser/
/renewable/
/renewable/afv/
/nuclear/
/nuclear/outages/
/totalenergy/
/totalenergy/data/monthly/
/totalenergy/data/annual/
/analysis/
/outlooks/steo/
/outlooks/aeo/
/outlooks/ieo/
/finance/
/finance/data.php
/finance/markets/crudeoil/
/environment/
/environment/emissions/carbon/
/environment/emissions/state/analysis/
/environment/emissions/state/
/special/disruptions/
/special/disruptions/
/state/
/state/seds/
/maps/
/state/maps.php
/special/gulf_of_mexico/
/beta/international/
/beta/i

In [20]:
base_url = 'https://www.eia.gov/electricity'
state_links = []
for i in soup.find_all('a'):
    url = i.get('href')
    if url and '../state/' in url and '../state/districtofcolumbia' not in url and '../state/unitedstates' not in url:
        new_url = url.replace('..', '')  #remove the '..' from the string
        state_links.append(new_url)
print(len(state_links))

50


In [21]:
## construct the state_links for a sample state and parse the html data

print(base_url + state_links[0]) # Test getting the data for one state
r = requests.get(base_url + state_links[0])
soup = BeautifulSoup(r.text, 'html.parser')


https://www.eia.gov/electricity/state/alabama


In [22]:
## The data we need is in tr tag

rows = soup.find_all('tr')
print(len(rows))

21


In [23]:
## we need the rows that contains 'Carbon dioxide (lbs/MWh)'
for row in rows:
    if 'Carbon dioxide (lbs/MWh)' in str(row):
        target_rows = row
        print(target_rows)


<tr>
<td>Carbon dioxide (lbs/MWh)</td>
<td>851</td>
<td>32</td>
</tr>


In [24]:
co2 = float(target_rows.find_all('td')[1].text.replace(',', ''))  #convert text to float, replace ',' with no space
print(co2)

851.0


In [25]:
## Get the state name, the state name is under the 'caption' tag
s =soup.find_all('caption')[0].string
print(s)

#get the text inside the bracket
state = s[s.find('(')+1 : s.find(')')]  #1 is added to avoid the first '('
print(state)


						Table 1. 2018 Summary statistics (Alabama) 					
Alabama


In [26]:
## make a data dictionary for the test state
data = {}
data[state] = co2
print(data)

{'Alabama': 851.0}


In [27]:
## new iterate for all the 50 states to get data
data = {}  #dictionary which will contain the state name and target dtaa

for link in state_links:
    r = requests.get(base_url + link)
    soup = BeautifulSoup(r.text, 'html.parser')
    rows = soup.find_all('tr')
    for row in rows:
        if 'Carbon dioxide (lbs/MWh)' in str(row):
            target_rows = row            
            co2 = float(target_rows.find_all('td')[1].text.replace(',', ''))  #convert text to float, replace ',' with no space
            s =soup.find_all('caption')[0].string
            state = s[s.find('(')+1 : s.find(')')]  #1 is added to avoid the first '('
            print(f'Accessing data for {state}..')
            data[state] = co2         
    
print('\nObtained data:')
print(data)

print(f'\nSuccessfully aobtained data from {len(data)} states')


Accessing data for Alabama..
Accessing data for Alaska..
Accessing data for Arizona..
Accessing data for Arkansas..
Accessing data for California..
Accessing data for Colorado..
Accessing data for Connecticut..
Accessing data for Delaware..
Accessing data for Florida..
Accessing data for Georgia..
Accessing data for Hawaii..
Accessing data for Idaho..
Accessing data for Illinois..
Accessing data for Indiana..
Accessing data for Iowa..
Accessing data for Kansas..
Accessing data for Kentucky..
Accessing data for Louisiana..
Accessing data for Maine..
Accessing data for Maryland..
Accessing data for Massachusetts..
Accessing data for Michigan..
Accessing data for Minnesota..
Accessing data for Mississippi..
Accessing data for Missouri..
Accessing data for Montana..
Accessing data for Nebraska..
Accessing data for Nevada..
Accessing data for New Hampshire..
Accessing data for New Jersey..
Accessing data for New Mexico..
Accessing data for New York..
Accessing data for North Carolina..
Acce

In [28]:
## there are some spelling mistakes in the website for 'Carbon dioxide' after South Dakota. So following section of codes is executed
for link in state_links[41:]:
    r = requests.get(base_url + link)
    soup = BeautifulSoup(r.text, 'html.parser')
    rows = soup.find_all('tr')
    for row in rows:
        if 'Carbon dioxide (lbs/MWh)' in str(row):
            target_rows = row            
            co2 = float(target_rows.find_all('td')[1].text.replace(',', ''))  #convert text to float, replace ',' with no space
            s =soup.find_all('caption')[0].string
            state = s[s.find('(')+1 : s.find(')')]  #1 is added to avoid the first '('
            print(f'Accessing data for {state}..')
            data[state] = co2
            
        if 'Carbon Dioxide (lbs/MWh)' in str(row):
            target_rows = row            
            co2 = float(target_rows.find_all('td')[1].text.replace(',', ''))  #convert text to float, replace ',' with no space
            s =soup.find_all('caption')[0].string
            state = s[s.find('(')+1 : s.find(')')]  #1 is added to avoid the first '('
            print(f'Accessing data for {state}..')
            data[state] = co2

        if 'Carbon sioxide (lbs/MWh)' in str(row):
            target_rows = row            
            co2 = float(target_rows.find_all('td')[1].text.replace(',', ''))  #convert text to float, replace ',' with no space
            s =soup.find_all('caption')[0].string
            state = s[s.find('(')+1 : s.find(')')]  #1 is added to avoid the first '('
            print(f'Accessing data for {state}..')
            data[state] = co2

    
print('\nObtained data:')
print(data)

print(f'\nSuccessfully aobtained data from {len(data)} states')

Accessing data for Tennessee..
Accessing data for Texas..
Accessing data for Utah..
Accessing data for Vermont..
Accessing data for Virginia..
Accessing data for Washington..
Accessing data for West Virginia..
Accessing data for Wisconsin..
Accessing data for Wyoming..

Obtained data:
{'Alabama': 851.0, 'Alaska': 1194.0, 'Arizona': 919.0, 'Arkansas': 1241.0, 'California': 491.0, 'Colorado': 1379.0, 'Connecticut': 535.0, 'Delaware': 1126.0, 'Florida': 971.0, 'Georgia': 933.0, 'Hawaii': 1616.0, 'Idaho': 214.0, 'Illinois': 846.0, 'Indiana': 1775.0, 'Iowa': 1189.0, 'Kansas': 1010.0, 'Kentucky': 1850.0, 'Louisiana': 1094.0, 'Maine': 429.0, 'Maryland': 896.0, 'Massachusetts': 808.0, 'Michigan': 1167.0, 'Minnesota': 1066.0, 'Mississippi': 907.0, 'Missouri': 1697.0, 'Montana': 1218.0, 'Nebraska': 1519.0, 'Nevada': 777.0, 'New Hampshire': 282.0, 'New Jersey': 554.0, 'New Mexico': 1242.0, 'New York': 464.0, 'North Carolina': 814.0, 'North Dakota': 1615.0, 'Ohio': 1361.0, 'Oklahoma': 880.0, 'Oreg

In [15]:
##save the data in .csv file

import csv

with open('CO2 lbs per MWh.csv','w') as f:
    w = csv.writer(f)
    w.writerows(data.items())